In [2]:
import numpy as np
import requests as rq

from fl.model import NN
from fl.preprocessing import load_mnist, data_to_client
from fl.federated_learning import train_and_test


URL = "http://localhost:8000/"
# URL = "https://du-poison.challenges.404ctf.fr/" 

# Solution du challenge 2 : Du poison [2/2]

Lors du 404 CTF, j'ai mis un seuil de 0.5 pour les tests afin d'être large et m'assurer que toute personne avec une solution décente réussisse à obtenir le drapeau à chaque coup. Malheureusement, il se trouve qu'il était beaucoup trop haut et que beaucoup de joueurs ont pu réussir le challenge 1 et 2 de la même façon. J'ai donc modifié le seuil à 0.3 ici, pour pouvoir présenter une solution différente. 

Comme avant on récupère le modèle, cette fois on va l'entraîner correctement : 

In [4]:
dataset = load_mnist()
model = NN()
model.load_weights("../weights/base_fl.weights.h5")
x_train, y_train, x_test, y_test = dataset
x_clients, y_clients = data_to_client(x_train, y_train)

In [5]:
results = train_and_test(model, x_clients[0], y_clients[0], x_test, y_test, adam_lr=0.04)
weights = results["model"].get_weights()

63/63 [==============================] - 0s 3ms/step
Accuracy of the model: 0.799


On réessaie la méthode du challenge 1 : 

In [6]:
d = {
    "w1": np.random.random(weights[0].shape).tolist(),
    "b1": np.random.random(weights[1].shape).tolist(),
    "w2": np.random.random(weights[2].shape).tolist(),
    "b2": np.random.random(weights[3].shape).tolist(),
    "w3": np.random.random(weights[4].shape).tolist(),
    "b3": np.random.random(weights[5].shape).tolist(),
    "w4": np.random.random(weights[6].shape).tolist(),
    "b4": np.random.random(weights[7].shape).tolist()
}

In [7]:
rq.get(URL + "healthcheck").json()

{'message': 'Statut : en pleine forme !'}

In [8]:
rq.post(URL + "challenges/2", json=d).json()

{'message': "Raté ! Le score de l'apprentissage fédéré est de 0.4055. Il faut l'empoisonner pour qu'il passe en dessous de 0.3"}

Comme mentionné précédemment, cette méthode fonctionnait durant la compétition (0.4055 < 0.5) mais ici vu que le seuil est à 0.3 il va falloir trouver autre chose. 

Que s'est-il passé ? 

Le challenge 1 ne comportait aucune protection. Du coup, lorsqu'on a mis des poids aléatoires prenant des valeurs entre -1 et 1, cela a complètement cassé le modèle. En effet, les valeurs usuelles des poids sont très proches de 0, de l'ordre de 0.001 pour les réseaux utilisés dans ces challenges. De ce fait, lors de la mise en commun, les poids aléatoires vont largement prendre le dessus, et empoisonner tout le modèle commun. 
Cette fois le challenge comporte une petite protection. Pour ne pas avoir de valeurs trop aberrantes, le serveur s'occupe d'abord de couper les poids au-dessus d'un certain seuil :
$$
w' = \text{sign}(w) \times \min(|w|, s)
$$

On cherche donc à avoir un impact maximal avec la plus petite amplitude de poids possible. On peut par exemple prendre l'inverse des poids calculés : on a calculé les poids pour maximiser l'augmentation de la précision du modèle, prendre l'inverse permet donc de maximiser la diminution de la précision du modèle.

In [12]:
d = {
    "w1": (-np.sign(weights[0])).tolist(),
    "b1": (-np.sign(weights[1])).tolist(),
    "w2": (-np.sign(weights[2])).tolist(),
    "b2": (-np.sign(weights[3])).tolist(),
    "w3": (-np.sign(weights[4])).tolist(),
    "b3": (-np.sign(weights[5])).tolist(),
    "w4": (-np.sign(weights[6])).tolist(),
    "b4": (-np.sign(weights[7])).tolist()
}

In [13]:
rq.post(URL + "challenges/2", json=d).json()

{'message': 'Bravo ! Voici le drapeau : 404CTF{p3rF0rm4nc3_Ou_s3cUR1T3_FaUt_iL_Ch01s1r?} (score : 0.261)'}